# IV. Fisher forecasts with `21cmfish`
This is a guide to reproduce our projected constraints in the papers 2312.11608 and 2509.22772.

First, clone this slightly modified fork of `21cmfish` at https://github.com/joshwfoster/21cmfish . Checkout the `master` branch (commit `9d63e65`). Install via `pip install .` in the project directory.

Next, create the lightcones necessary to do a fisher information forecast. This can be done by calling `scripts/inj_script.py`. Set the environment variable `DM21CM_OUTPUT_DIR` to your desired output directory.
For astrophysical background variations, in the `sciprts` directory, run
```bash
python inj_script.py --run_name bkg-run0 --channel bkg -i $I # I runs from 0-24, 0 is fiducial, 1-24 are shifts.
```
For dark matter injection, e.g. dark matter annihilating to photons via $p$-wave, run
```bash
python inj_script.py --run_name pwave-phot-run0 --channel pwave-phot -i $I # See inj_script.py for details on I.
```
The file `inj_script.py` and referenced files contains the default parameters such as dark matter mass, injection strength use in our production runs.

At this point, your `DM21CM_OUTPUT_DIR` should look like this:

```text
$DM21CM_OUTPUT_DIR
├── bkg-run0/
│   ├── LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_ALPHA_ESC_-0.03_r54321.h5
│   ├── LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_ALPHA_ESC_0.03_r54321.h5
│   ├── LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_ALPHA_STAR_-0.03_r54321.h5
│   │   ...
│   └── LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_ALPHA_STAR_-0.03_r54321.h5
└── pwave-phot-run0/
    ├── log10m1.500/
    │   ├── LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_DM_1_r54321.h5
    │   └── LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_DM_2_r54321.h5 # More if you are using higher order stencils
    ├── log10m2.000/
    │   ├── LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_DM_1_r54321.h5
    │   └── LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_DM_2_r54321.h5
    ├── ...
```

You can now follow the rest of the notebook.


In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import shutil

import numpy as np
from scipy import stats
from tqdm import tqdm
from IPython.display import clear_output
import re

import py21cmfish
from py21cmfish.power_spectra import *
from py21cmfish.io import *

import matplotlib as mpl
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
mpl.rc_file("../matplotlibrc")

## 1. Build fisher matrix for astrophysics variations

In [ ]:
bkg_dir = os.environ['DM21CM_OUTPUT_DIR'] + "/bkg-run0/"
PS_err_dir = os.environ['DM21CM_DIR'] + '/data/21cmSense_fid_EOS21/'

astro_params_vary = ['DM', 'F_STAR10', 'F_STAR7_MINI', 'ALPHA_STAR', 'ALPHA_STAR_MINI', 't_STAR',
                     'F_ESC10', 'F_ESC7_MINI', 'ALPHA_ESC', 'L_X', 'L_X_MINI', 'NU_X_THRESH', 'A_LW']
default_param_values = [0, -1.25, -2.5, 0.5, 0.0, 0.5, -1.35, -1.35, -0.3, 40.5, 40.5, 500, 2.0]

astro_params_fid = dict()
for i, ap in enumerate(astro_params_vary):
    astro_params_fid[ap] = default_param_values[i]

# Load each parameter into a dictionary
params_EoS = {}
for param in astro_params_vary[1:]:
    params_EoS[param] = py21cmfish.Parameter(
        HII_DIM=128, BOX_LEN=256, param=param,
        output_dir = bkg_dir,
        PS_err_dir = PS_err_dir,
        new = False
)
clear_output()

## 2. Fisher forecast for each mass

In [ ]:
run_name = 'pwave-phot-run0'
channel = run_name.rsplit('-', 1)[0]
inj_dir = os.environ['DM21CM_OUTPUT_DIR'] + f"/{run_name}"
print(os.listdir(inj_dir))

log10m_s = np.sort([
    float(re.match(r'log10m([-\d\.]+)', d).group(1))
    for d in os.listdir(inj_dir)
    if re.match(r'log10m([-\d\.]+)', d)
])
m_s = 10**log10m_s
print('log10m_s', log10m_s)

EPSILON = 1e-6

ss = StepSize250909()
if channel == 'decay-phot':
    tau_s = ss.decay_phot_lifetime(m_s)
    inj_s = 1/tau_s
elif channel == 'decay-elec':
    tau_s = ss.decay_elec_lifetime(m_s)
    inj_s = 1/tau_s
elif channel.startswith('pwave-phot'):
    c_s = ss.pwave_phot_c_sigma(m_s)
    inj_s = c_s
elif channel == 'pwave-elec':
    c_s = ss.pwave_elec_c_sigma(m_s)
    inj_s = c_s
elif channel == 'pwave-tau':
    c_s = ss.pwave_tau_c_sigma(m_s)
    inj_s = c_s
elif channel.startswith('pbhhr'):
    a_PBH = float(channel.split('-')[1][1:])
    f_s = ss.pbhhr_f(m_s, a=a_PBH)
    inj_s = f_s
elif channel.startswith('pbhacc'):
    model = channel.split('-')[1]
    f_s = ss.pbhacc_f(m_s, model)
    inj_s = f_s

# Copy the fiducial lightcone in each mass directory
print('Copied :', end=' ')
for m in m_s:
    source_file = f'{bkg_dir}/LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_fid_r54321.h5'
    m_dir = f'log10m{np.log10(m):.3f}'
    target_file = f'{inj_dir}/{m_dir}/LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_fid_r54321.h5'
    if not os.path.isfile(target_file):
        print(f'{np.log10(m):.3f}', end=' ')
        shutil.copyfile(source_file, target_file)

In [ ]:
sigma_s = []
force_new = False

for m in tqdm(m_s):

    lc_dir = f'{inj_dir}/log10m{np.log10(m):.3f}/'
    new = ('lc_redshifts.npy' not in os.listdir(lc_dir)) or force_new
    
    for param in astro_params_vary[:1]:
        params_EoS[param] = py21cmfish.Parameter(
            HII_DIM=128, BOX_LEN=256, param=param,
            output_dir=lc_dir,
            PS_err_dir=PS_err_dir,
            new=new,
            dm_deriv_order=2
        )

    Fij_matrix_PS, Finv_PS = py21cmfish.make_fisher_matrix(
        params_EoS,
        fisher_params=astro_params_vary,
        hpeak=0.0, obs='PS',
        k_min=0.1, k_max=1,
        sigma_mod_frac=0.2,
        add_sigma_poisson=True
    )
    sigma_s.append(np.sqrt(Finv_PS[0, 0]))
    
sigma_s = np.array(sigma_s)
print('sigma', sigma_s)

# 3. Plotting and saving the results

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

one_sigma = inj_s * sigma_s
upper_limit = np.sqrt(stats.chi2.ppf(.9, df=1)) * one_sigma

ax.plot(m_s, upper_limit, 'k-s', label='limit')
ax.fill_between(m_s, upper_limit + one_sigma, upper_limit - one_sigma, color=mpl.colormaps['viridis'](0.75), label='1$\sigma$')
ax.fill_between(m_s, upper_limit + 2*one_sigma, upper_limit + one_sigma, color=mpl.colormaps['viridis'](1.0), label='2$\sigma$')

ax.set(xscale='log', yscale='log')
ax.set(title=run_name)

m_fine_s = np.geomspace(m_s[0], m_s[-1], 100)
if channel == 'decay-phot':
    ax.plot(m_fine_s, 1/ss.decay_phot_lifetime(m_fine_s), 'r--', label='step size')
    ax.set(xlabel=r'$m_\chi$ [eV]', ylabel=r'$1/\tau$ [s$^{-1}$]')
elif channel == 'decay-elec':
    ax.plot(m_fine_s, 1/ss.decay_elec_lifetime(m_fine_s), 'r--', label='step size')
    ax.set(xlabel=r'$m_\chi$ [eV]', ylabel=r'$1/\tau$ [s$^{-1}$]')
elif channel == 'pwave-phot':
    ax.plot(m_fine_s, ss.pwave_phot_c_sigma(m_fine_s), 'r--', label='step size')
    ax.set(xlabel=r'$m_\chi$ [eV]', ylabel=r'$C_\sigma$ [cm$^3$/s]')
elif channel == 'pwave-elec':
    ax.plot(m_fine_s, ss.pwave_elec_c_sigma(m_fine_s), 'r--', label='step size')
    ax.set(xlabel=r'$m_\chi$ [eV]', ylabel=r'$C_\sigma$ [cm$^3$/s]')
elif channel == 'pwave-tau':
    ax.plot(m_fine_s, ss.pwave_tau_c_sigma(m_fine_s), 'r--', label='step size')
    ax.set(xlabel=r'$m_\chi$ [eV]', ylabel=r'$C_\sigma$ [cm$^3$/s]')
elif channel.startswith('pbhhr'):
    ax.plot(m_fine_s, ss.pbhhr_f(m_fine_s, a=a_PBH), 'r--', label='step size')
    ax.set(xlabel=r'$M$ [g]', ylabel=r'$f$')
elif channel.startswith('pbhacc'):
    ax.plot(m_fine_s, ss.pbhacc_f(m_fine_s, model), 'r--', label='step size')
    ax.set(xlabel=r'$M$ [Msun]', ylabel=r'$f$')
ax.legend()

In [ ]:
save_fn = os.environ['DM21CM_DIR'] + f'/outputs/limits/{run_name}.txt'
dir_path = os.path.dirname(save_fn)
os.makedirs(dir_path, exist_ok=True)
np.savetxt(save_fn, np.array([m_s, inj_s, sigma_s]).T, header='mass_s inj_s sigma_s')